# 1.0 Let's Build a ReAct Agent with Python from Scratch

## **Introduction**

Welcome to this hands-on workshop! In this session, we will build a ReAct-based AI Agent from scratch using Python. The ReAct framework combines reasoning and acting, enabling the creation of intelligent agents that can tackle dynamic tasks. 

By the end of this workshop, you will:
- Build a fully functional ReAct AI agent.
- Learn how to incorporate reasoning and actions into the agent’s decision-making process.
- Gain hands-on experience in building agents that autonomously interact with dynamic environments.

## **Workshop Outline**
1. Understand the ReAct framework and its components.
2. Develop the core structure of the AI agent.
3. Implement reasoning and actions within the agent.
4. Test the agent's functionality in a practical scenario.

🚀 Let's get started and build your first ReAct agent!

# Introduction to React Agents

## What are React Agents?

React (Reasoning + Acting) Agents are a type of AI agent designed to dynamically reason about tasks and take actions based on the environment and available tools. 
Unlike traditional static AI models, React Agents combine **thought generation** with **action execution**, iteratively refining their decisions using feedback from observations.

React Agents are particularly useful in tasks requiring **multi-step reasoning**, **tool usage**, and **adaptive decision-making**, making them ideal for applications such as automated assistants, network configuration, and problem-solving systems.

## Key Features of React Agents

- **Iterative Thought and Action Loop**: The agent cycles through **thought, action, observation, and reasoning** to improve decision-making.
- **Tool Use and Interaction**: It can invoke external tools such as databases, APIs, and computational functions to obtain necessary information.
- **Self-Correction and Adaptability**: React Agents refine their approach based on real-time feedback, making them more efficient in complex environments.


![React Agent Diagram](images/react-agent.png)

### **Complete the Following Pre-Requisites**  
1. Select the kernel: **"Python(ai-agent)"**  
2. Perform **Clear all outputs**

### Step 1: Importing Required Libraries and Loading Environment Variables

In this first cell, we are setting up the necessary libraries to interact with the OpenAI API and handle environment variables securely. Let’s break down what each line does:

1. **Import Libraries**:
   - `openai`: The Python client for interacting with the OpenAI API. We’ll use it to send requests to the OpenAI API and generate text completions.
   - `re`: A library for working with regular expressions. It helps us manipulate strings and perform complex string matching operations.
   - `httpx`: An HTTP client that supports asynchronous requests, useful for making API calls.
   - `os`: Provides functions for interacting with the operating system, including working with environment variables.
   - `dotenv`: A module to load environment variables from a `.env` file, which is useful for storing sensitive information like API keys securely.

2. **Loading Environment Variables**:
   - `load_dotenv()`: This function loads variables from a `.env` file into the environment. It allows us to store sensitive information, like our OpenAI API key, outside of the code, keeping it secure and preventing accidental exposure.

3. **Importing OpenAI Class**:
   - `from openai import OpenAI`: This imports the necessary components from the OpenAI Python client to interact with the API.

#### By running this code, we are setting up the tools and configurations required to make requests to the OpenAI API in a secure and efficient manner.


In [ ]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv
_ = load_dotenv()
from openai import OpenAI

### Step 2: Testing LLM Calls

Now, we are testing whether our LLM (Large Language Model) calls are working. This step sends a simple message to the OpenAI API and retrieves the response.

1. **Creating the OpenAI Client**:
   - `client = OpenAI()`: This line initializes the OpenAI client, which will be used to interact with the OpenAI API.

2. **Sending a Chat Message**:
   - `chat_completion = client.chat.completions.create(...)`: This line sends a request to the OpenAI API to generate a response based on the given input. 
   - Here, we specify:
     - `model="gpt-3.5-turbo"`: We choose the GPT-3.5 turbo model, which is optimized for chat and conversational tasks.
     - `messages=[{"role":"user", "content":"Hello world"}]`: We send a message from the `"user"` role with the content `"Hello world"`. This is the input that the model will respond to.

3. **Extracting the Response**:
   - `chat_completion.choices[0].message.content`: This accesses the first choice from the model’s response and retrieves the `content`, which contains the model's generated reply.

#### This step ensures that our integration with the OpenAI API is working and that we can successfully send inputs and receive outputs from the model.


In [ ]:
client = OpenAI()

chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role":"user", "content":"Hello world"}]
)

chat_completion.choices[0].message.content
    


### Step 3: Building an Agent

In this part of the notebook, we are building an `Agent` class, which will interact with the OpenAI API to send and receive messages in a conversational manner. This agent is designed to manage context, handle user input, and generate appropriate responses based on the conversation.

1. **Creating the Agent Class**:
   - `class Agent:`: We define a class called `Agent`. This will serve as the core structure for managing a conversation with the model.

2. **Initialization Method (`__init__`)**:
   - The __init__ method initializes the agent with an optional system message for context or instructions. It stores this message in `self.system` and initializes an empty list `self.messages` to hold the conversation. If a system message is provided, it's added to the messages list with the "system" role to set up agent behavior or provide context

3. **Calling the Agent (`__call__` Method)**:
   - The `__call__` method allows the `Agent` class to be used like a function. When called, it adds the user’s message to `self.messages`, invokes the `execute()` method to get a response from the OpenAI API, appends the response to `self.messages` with the `"assistant"` role, and returns the assistant's reply.

4. **Execute Method (`execute`)**:
   - The `execute` method sends the conversation to the OpenAI API and retrieves the assistant's response. It uses the `gpt-4` model with a `temperature` of 0 for focused and predictable replies, and sends the full conversation history. The method then returns the assistant's response.


In [ ]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

### Step 4: Structured Problem-Solving Loop for the Agent

In this section, we define a **prompt** that guides the agent through a structured problem-solving loop. The agent will run through a sequence of steps to answer a given question, ensuring the process is logical and methodical.

1. **The Loop**:
   - **Thought**: The agent starts by reflecting on the question it has been asked. This is the agent’s internal step of analyzing what it knows and deciding how to proceed with answering the question.
   - **Action**: Based on the thought, the agent will perform one of the available actions. Actions include running calculations or looking up specific information.
   - **PAUSE**: After executing an action, the agent pauses and waits for the result, allowing time for the action to complete and provide an output.
   - **Observation**: The result of the action is observed and returned to the agent. The observation informs the agent’s next steps.
   - **Answer**: Finally, after receiving the observation, the agent uses the information to provide a clear and concise answer to the original question.

2. **Available Actions**:
   - **calculate**: The agent can perform calculations using Python syntax. For example, `calculate: 4 * 7 / 3` will return the result of this mathematical operation.
   - **average_elephant_weight**: This action allows the agent to retrieve the average weight of an elephant species based on the species name. For example, `average_elephant_weight: African Elephant` will return the average weight of an African Elephant.

3. **Example Session**:
   - **Question**: "How much does an African Elephant weigh?"
   - **Thought**: The agent decides it needs to look up the weight of the elephant.
   - **Action**: The agent performs the action `average_elephant_weight: African Elephant` to find the weight of an African Elephant.
   - **PAUSE**: The agent waits for the response from the action.
   - **Observation**: The result is received: "An African Elephant weighs 12,000 lbs."
   - **Answer**: The agent then outputs: "An African Elephant weighs 12,000 lbs."

#### This prompt structure creates a step-by-step framework for the agent to reason, act, pause, and observe in order to arrive at a final answer. It ensures that the agent logically analyzes the problem, executes appropriate actions, and synthesizes the information to generate accurate answers.


In [ ]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_elephant_weight:
e.g. average_elephant_weight: African Elephant
Returns the average weight of an elephant species when given the species name.

Example session:

Question: How much does an African Elephant weigh?
Thought: I should look up the elephant's weight using average_elephant_weight.
Action: average_elephant_weight: African Elephant
PAUSE

You will be called again with this:

Observation: An African Elephant weighs 12,000 lbs.

You then output:

Answer: An African Elephant weighs 12,000 lbs.
""".strip()

### Step 5: Defining Actions for the Agent

1. **`calculate(what)`**:
   - Evaluates a mathematical expression passed as a string using Python’s `eval()` function and returns the result.

2. **`average_elephant_weight(species)`**:
   - Looks up the average weight of an elephant species from a predefined dictionary and returns it. If the species is not found, it returns `"Unknown elephant species"`.

3. **`known_actions`**:
   - A dictionary that maps action names (`"calculate"`, `"average_elephant_weight"`) to their respective functions, enabling the agent to execute them based on the task.

#### These functions define the actions available to the agent, and the `known_actions` dictionary links action names to their implementations.


In [ ]:
def calculate(what):
    return eval(what)

def average_elephant_weight(species):
    weights = {
        "african elephant": "An African Elephant weighs 12,000 lbs",
        "asian elephant": "An Asian Elephant weighs 8,800 lbs",
        "forest elephant": "A Forest Elephant weighs 6,000 lbs",
        "pygmy elephant": "A Pygmy Elephant weighs 5,500 lbs",
        "indian elephant": "An Indian Elephant weighs 9,000 lbs",
        "sumatran elephant": "A Sumatran Elephant weighs 6,600 lbs",
        "sri lankan elephant": "A Sri Lankan Elephant weighs 10,000 lbs"
    }
    return weights.get(species.lower(), "Unknown elephant species")

known_actions = {
    "calculate": calculate,
    "average_elephant_weight": average_elephant_weight
}

### Step 6: Initializing the Agent and Interacting with the Agent

- **`abot = Agent(prompt)`**:
   - This line creates an instance of the `Agent` class, named `abot`, and initializes it with the predefined `prompt` that guides the agent’s thought process, actions, and responses.

#### The `Agent` instance (`abot`) is now ready to use the defined prompt to interact and perform actions based on user queries.

- **`result = abot("How much does an African elephant weigh?")`**:
   - This line sends the question "How much does an African elephant weigh?" to the `abot` instance (the agent) and stores the response in the `result` variable.

- **`print(result)`**:
   - This prints the agent’s response, which will include the reasoning and the final answer to the question.

#### The agent processes the question, runs the necessary actions, and outputs the answer, which is then printed to the console.

In [ ]:
abot = Agent(prompt)
result = abot("How much does a African elephant weigh?")
print(result)

### Summarizing the Agent's Interaction

- **`result = average_elephant_weight("African Elephant")`**:
   - This line calls the `average_elephant_weight` function with the argument `"African Elephant"`, which returns the average weight of an African elephant and stores it in the `result` variable.

- **`print(result)`**:
   - This prints the result, which is the average weight of the African elephant.

- **`next_prompt = "Observation: {}".format(result)`**:
   - This formats the result into a string and stores it in `next_prompt`, which includes the observation based on the elephant's weight.

- **`abot(next_prompt)`**:
   - This sends the formatted `next_prompt` to the agent `abot`, allowing it to process the observation and respond accordingly.

#### The agent processes the observation and generates a response based on the information provided.


In [ ]:
result = average_elephant_weight("African Elephant")
print(result)
next_prompt = "Observation: {}".format(result)
abot(next_prompt)

#### This code allows you to view the entire conversation history stored in `abot.messages`, which tracks the agent's state and context during the interaction.

In [ ]:
abot.messages

### Step 7: Defining a Regular Expression to Select Action

- **`action_re = re.compile(r'^Action: (\w+): (.*)$')`**:
   - This regular expression captures actions in the format `Action: <action_name>: <details>`, extracting the action type and its details for dynamic processing.

#### It enables the agent to process different actions based on the captured details.


In [ ]:
action_re = re.compile(r'^Action: (\w+): (.*)$')   # python regular expression to selection action

### Step 8: Defining the `query` Function

- **`query(question, max_turns=5)`**:
   - This function simulates a conversation with the agent, taking a `question` as input and running up to `max_turns` (default 5) to process the response and actions.
   - **Loop**:
     - The loop iterates up to `max_turns`. For each iteration, the agent responds to the current `next_prompt`, which initially starts as the question.
     - The response (`result`) is printed, and the function looks for actions in the response using the regular expression `action_re`.
     - If an action is found, the function runs the corresponding action and retrieves the observation.
     - The `next_prompt` is updated with the observation and the loop continues.
   - If no action is found, the function exits.

#### The `query` function sends a question to the agent, processes the agent's responses and actions, and continues until the maximum number of turns is reached or no actions are left to process.


In [ ]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(f" -- running {action} {action_input}")
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return

#### The `query` function is used to ask the agent a question about the combined weight of two elephants, and the agent processes the question step-by-step, executing any actions required to compute the answer.


In [ ]:
question = """I have 2 elephants in the zoo, a African Elephant and a Sri lankan ELephant. \
What is their combined weight"""
query(question)

## Conclusion

#### In this tutorial, we learned how to build a simple reactive agent using Python. The agent processes user input through a structured loop of Thought, Action, and Observation, enabling it to reason and perform tasks. We defined actions such as calculations and retrieving specific data (e.g., elephant weights). The agent dynamically interacts with these actions and updates its state to answer complex questions. This approach allows the agent to handle various tasks effectively by combining reasoning with real-time execution of actions.


## Reference

#### React Agents were first introduced in the paper:

**"ReAct: Synergizing Reasoning and Acting in Language Models"**  
Authors: Shunyu Yao, Jeffrey Zhao, Dian Yu, Nan Du, Izhak Shafran, Thomas L. Griffiths, Karthik Narasimhan  
Arxiv: [https://arxiv.org/abs/2210.03629](https://arxiv.org/abs/2210.03629)  

#### This paper outlines the principles of React Agents and demonstrates their effectiveness in various reasoning and decision-making tasks.